<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Zalocusky_Data_Scanpy_MERGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy

In [ ]:
import h5py
import numpy as np
import scipy as sci
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd


sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.3 anndata==0.9.1 umap==0.5.3 numpy==1.23.5 scipy==1.10.0 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 pynndescent==0.5.10


In [ ]:
def trim_key(k):
    k = k.replace('_raw_gene_bc_matrices_h5.h5',"")
    k = k.replace('_raw_gene_bc_matrics_h5.h5',"")
    k =k.replace(k.split("_")[0],"")
    return k

In [ ]:
path = 'C:/Users/dennis.tabuena/Downloads/GSE167497_RAW (2).tar'
my_tar = tarfile.open(path)
my_tar.extractall('./my_folder') # specify which folder to extract to
my_tar.close()
for f in os.listdir('./my_folder'):
    print(f)

GSM5106143_02_E4_14_H0_raw_gene_bc_matrices_h5.h5
GSM5106144_03_E3_10_H10_raw_gene_bc_matrics_h5.h5
GSM5106145_04_E4_10_H11_raw_gene_bc_matrices_h5.h5
GSM5106146_05_E3_14_H12_raw_gene_bc_matrices_h5.h5
GSM5106147_06_E3_5_A01_raw_gene_bc_matrices_h5.h5
GSM5106148_07_E4_5_A02_raw_gene_bc_matrices_h5.h5
GSM5106149_08_E3_10_A03_raw_gene_bc_matrices_h5.h5
GSM5106150_09_E4_10_A04_raw_gene_bc_matrices_h5.h5
GSM5106151_10_E3_10_A05_raw_gene_bc_matrices_h5.h5
GSM5106152_11_E4_10_A06_raw_gene_bc_matrices_h5.h5
GSM5106153_12_E3_20_A07_raw_gene_bc_matrices_h5.h5
GSM5106154_13_E4_20_A08_raw_gene_bc_matrices_h5.h5
GSM5106155_14_E3_20_A09_raw_gene_bc_matrices_h5.h5
GSM5106156_15_E4_20_A10_raw_gene_bc_matrices_h5.h5
GSM5106157_16_E3_15_A11_raw_gene_bc_matrices_h5.h5
GSM5106158_17_E4_15_A12_raw_gene_bc_matrices_h5.h5
GSM5106159_18_E3_15_B01_raw_gene_bc_matrices_h5.h5
GSM5106160_19_E4_15_B02_raw_gene_bc_matrices_h5.h5
GSM5106161_20_E3_5_B03_raw_gene_bc_matrices_h5.h5
GSM5106162_21_E4_5_B04_raw_gene_bc_m

In [ ]:
adata_dict = {}
for f in os.listdir('./my_folder'):
    if 'YH' not in f:
        a = sc.read_10x_h5('./my_folder/'+f)
        a.var_names_make_unique()
        adata_dict[trim_key(f)] = a

C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\anndata\_core\

In [ ]:
adata = ad.concat(adata_dict,axis = 0,label="Sample",index_unique="_")
display(adata.obs.head())


,Sample
AAACCTGAGAAACCAT-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACCGC-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACCTA-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACGAG-1__02_E4_14_H0,_02_E4_14_H0
AAACCTGAGAAACGCC-1__02_E4_14_H0,_02_E4_14_H0


In [ ]:
adata_filt = adata.copy()

adata_filt.var['mt'] = adata_filt.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata_filt, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata_filt = adata_filt[adata_filt.obs.pct_counts_mt < 0.25, :]
sc.pp.filter_cells(adata_filt, min_genes=475)
sc.pp.filter_cells(adata_filt, max_genes=4000)
sc.pp.filter_genes(adata_filt, min_cells=1)
sc.pp.filter_genes(adata_filt, max_cells=np.inf)
display(adata_filt)


View of AnnData object with n_obs × n_vars = 14297602 × 27998
    obs: 'Sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

filtered out 14159293 cells that have less than 475 genes expressed


C:\Users\dennis.tabuena\AppData\Local\anaconda3\lib\site-packages\scanpy\preprocessing\_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


filtered out 168 cells that have more than 4000 genes expressed
filtered out 4470 genes that are detected in less than 1 cells


AnnData object with n_obs × n_vars = 138141 × 23528
    obs: 'Sample', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'